In [ ]:
%load_ext autoreload
%autoreload 2

import dill
import numpy as np
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTROL_DT, PITCH_DT, CONTACT_DT, BLENDER_DT, NUM_JOINTS
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS
from iiwa_batter.physics import find_ball_initial_velocity, PITCH_START_POSITION, FLIGHT_TIME_MULTIPLE

from iiwa_batter.swing_simulator import setup_simulator, run_swing_simulation, reset_systems
from iiwa_batter.plotting import plot_joint_velocity
from iiwa_batter.trajectory_library import (
    Trajectory, 
    LIBRARY_SPEEDS_MPH, 
    LIBRARY_POSITIONS,
    MAIN_SPEED,
    MAIN_POSITION,
)
from iiwa_batter.swing_optimization.stochastic_gradient_descent import (
    make_torque_trajectory,
    expand_torque_trajectory,
)
from iiwa_batter.export_for_blender import export_keyframes
from iiwa_batter.real_time_operation import real_time_operation

In [ ]:
meshcat = StartMeshcat()

In [ ]:
# Demonstration of the iiwa14 320 Nm joint able to spin up very fast

simulator, diagram = setup_simulator(
    torque_trajectory={0: np.array([320, 0, 0, 0, 0, 0, 0])},
    model_urdf="iiwa14_joint_1_only",
    dt=PITCH_DT, 
    meshcat=meshcat, 
    add_contact=False, 
)

status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=1,
    # MUST MATCH THE URDF!!!
    initial_joint_positions=np.array([0, np.pi/2, 0, 0, 0, -np.pi/2, np.pi]),
    initial_joint_velocities=np.array([0]*NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=np.zeros(3),
    meshcat=meshcat,
    record_state=True,
    check_dt=BLENDER_DT,
)

export_keyframes(status_dict, "iiwa14_torque_demo_no_cap")

In [ ]:
# Demonstration of the iiwa14 320 Nm joint getting speed capped
robot_constraints = JOINT_CONSTRAINTS["iiwa14"]

simulator, diagram = setup_simulator(
    torque_trajectory={0: np.array([200, 0, 0, 0, 0, 0, 0])}, 
    model_urdf="iiwa14_joint_1_only",
    dt=PITCH_DT, 
    meshcat=meshcat, 
    add_contact=False, 
    robot_constraints=robot_constraints)

results_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=0.2,
    # MUST MATCH THE URDF!!!
    initial_joint_positions=np.array([0, np.pi/2, 0, 0, 0, -np.pi/2, np.pi]),
    initial_joint_velocities=np.array([0]*NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=np.zeros(3),
    meshcat=meshcat,
    record_state = True
)

plot_joint_velocity(results_dict["state"], robot_constraints, f"{PACKAGE_ROOT}/../notebook_images/capped_velocity.png")



In [ ]:
# Demonstration of the iiwa14 40 Nm joint getting speed capped

In [ ]:
# Demonstration of searching for initial positions
robot_constraints = JOINT_CONSTRAINTS["iiwa14"]
simulator, diagram = setup_simulator(torque_trajectory={}, model_urdf="iiwa14", dt=PITCH_DT, add_contact=True, meshcat=meshcat)
import time

from iiwa_batter.swing_optimization.stochastic_gradient_descent import find_initial_positions
while True:
    initial_joint_positions = find_initial_positions(simulator, diagram, robot_constraints, 1)

    reset_systems(diagram)

    run_swing_simulation(
        simulator=simulator,
        diagram=diagram,
        start_time=0,
        end_time=2*CONTACT_DT,
        initial_joint_positions=np.zeros(NUM_JOINTS),
        initial_joint_velocities=np.zeros(NUM_JOINTS),
        initial_ball_position=PITCH_START_POSITION,
        initial_ball_velocity=np.zeros(3),
        meshcat=meshcat,
    )
    time.sleep(2)

In [ ]:
# Demonstration of the slugger old main swing hitting a ball

trajectory_loader = Trajectory("slugger", 90, LIBRARY_POSITIONS[0], "main")
initial_joint_positions, control_vector, _ = trajectory_loader.load_best_trajectory()

pitch_velocity, ball_time_of_flight = find_ball_initial_velocity(90, LIBRARY_POSITIONS[0])

trajectory = make_torque_trajectory(control_vector, ball_time_of_flight)

simulator, diagram = setup_simulator(
    torque_trajectory=trajectory,
    model_urdf="slugger",
    dt=CONTACT_DT, 
    meshcat=meshcat, 
    add_contact=True,
)

status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=0.6,
    initial_joint_positions=initial_joint_positions,
    initial_joint_velocities=np.zeros(NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=pitch_velocity,
    meshcat=meshcat,
    record_state=True,
    check_dt=BLENDER_DT,
)

export_keyframes(status_dict, "slugger_old_main_swing", extend_duration=True)

In [ ]:
# Demonstration of original main swing missing a pitch
pitch_speed = 85
target_position = [0, -0.1, 0.72]
actual_initial_velocity, actual_time_of_flight = find_ball_initial_velocity(pitch_speed, target_position)
closest_speed = min(LIBRARY_SPEEDS_MPH, key=lambda x: abs(x - pitch_speed))
closest_position = min(LIBRARY_POSITIONS, key=lambda x: np.linalg.norm(np.array(x) - np.array(target_position)))

robot = "iiwa14"
start_position_loader = Trajectory(robot, MAIN_SPEED, MAIN_POSITION, "main")
main_initial_position, _, _ = start_position_loader.load_best_trajectory()
start_trajectory_loader = Trajectory(robot, closest_speed, closest_position, "tune_fine")
_, start_control_vector, _ = start_trajectory_loader.load_best_trajectory()
_, closest_time_of_flight = find_ball_initial_velocity(closest_speed, closest_position)

planned_trajectory = make_torque_trajectory(start_control_vector, closest_time_of_flight)
planned_trajectory = expand_torque_trajectory(planned_trajectory, actual_time_of_flight+3*CONTROL_DT)

simulator, diagram = setup_simulator(
    torque_trajectory=planned_trajectory,
    model_urdf=robot,
    dt=CONTACT_DT,
    meshcat=meshcat, 
    add_contact=True,
    robot_constraints=JOINT_CONSTRAINTS[robot],
)

status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=actual_time_of_flight+3*CONTROL_DT,
    initial_joint_positions=main_initial_position,
    initial_joint_velocities=np.zeros(NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=actual_initial_velocity,
    meshcat=meshcat,
)

print(status_dict)

In [ ]:
# Demonstration of adapted main swing hitting a ball
taken_trajectory, status_dict = real_time_operation(
    robot=robot,
    pitch_speed_world = pitch_speed,
    pitch_position_world = target_position,
    pitch_speed_measurement_error=0,
    pitch_position_measurement_error=0,
)

In [ ]:
reset_systems(diagram, taken_trajectory)
status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=actual_time_of_flight*FLIGHT_TIME_MULTIPLE+CONTROL_DT,
    initial_joint_positions=main_initial_position,
    initial_joint_velocities=np.zeros(NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=actual_initial_velocity,
    meshcat=meshcat,
)

print(status_dict)